# 深層学習 day1-day2 実装

## 概要
ここでは、day1に関わる項目を復習しながら、全結合層を実装していきます。
ゼロかつくるディープラーニングを参考にしながら、2つの実装を行なっていきます。
主にday1-day2に跨る内容(CNNや最適化以外)を復習していきます。(day2はもう1つ実装演習をしています)

### 事前準備

In [1]:
import csv
import os
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split


# 乱数シードを指定
np.random.seed(seed=0)

### データの読み込み・線形結合層などの定義

3_DNN_day1で実装した関数やクラスを流用します。
同じくMNISTを使用します。

In [2]:
# mnistデータセットのロード(ネットワーク接続が必要・少し時間がかかります)
if os.path.exists('mnist_784'):
    with open('mnist_784','rb') as f:
        mnist = pickle.load(f)
else:
    mnist = datasets.fetch_openml('mnist_784')
    with open('mnist_784', 'wb') as f:
        pickle.dump(mnist, f)
    
# 画像とラベルを取得
X, T = mnist.data, mnist.target
# 訓練データとテストデータに分割
X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.2)
# ラベルデータをint型にし、one-hot-vectorに変換します
T_train = np.eye(10)[T_train.astype("int")]
T_test = np.eye(10)[T_test.astype("int")]

In [3]:
class FullyConnectedLayer():
    def __init__(self, input_shape, output_shape):
        self.w = np.random.randn(input_shape, output_shape) * 0.01
        self.b = np.zeros(output_shape, dtype=np.float)
        self.x = None
        self.dw = None
        self.db = None
        
    def __call__(self, x):
        self.x = x
        out = np.dot(self.x, self.w) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.w.T)
        batch_size = dx.shape[0]
        self.dw = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

In [4]:
class SoftmaxCrossEntropyLoss():
    def __init__(self):
        self.y = None
        self.t = None
        self.loss = None
        
    def __call__(self, t, y):
        self.y = softmax(y)
        self.t = t.copy()
        self.loss = cross_entropy_error(self.t, self.y)
        return self.loss
    
    def backward(self):
        batch_size = self.t.shape[0]
        dy = self.y - self.t
        dy /= batch_size
        return dy

In [5]:
def softmax(x):
    x = x.T
    _x = x - np.max(x, axis=0)
    _x = np.exp(_x) / np.sum(np.exp(_x), axis=0)
    return _x.T

In [6]:
def cross_entropy_error(t, y):
    delta = 1e-8
    error = -np.mean(t * np.log(y + delta))
    return error

In [7]:
class ReLU():
    def __init__(self):
        self.mask = None

    def __call__(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

### ドロップアウト

1. <font color="Red">ドロップアウトクラスを実装します。</font>
    - 入力されてきた ```x``` に対し、確率 ```dropout_ratio``` で出力を0にする。

In [8]:
class Dropout():
    def __init__(self, dropout_ratio=0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None

    def __call__(self, x, train_flg=True):
        if train_flg:
            randommatrix = np.random.rand(*x.shape)
            self.mask = randommatrix > self.dropout_ratio
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio)

    def backward(self, dout):
        return dout * self.mask
    
#参考 https://qiita.com/ishikawa-takumi/items/598dcd185a49b02883d5

### パラメータノルムペナルティ

2-1. <font color="Red">ベクトル・行列に対してLpノルムを計算する関数```lp_norm```を実装します。</font>

In [9]:
def lp_norm(x, p=2):
    return np.sum(np.abs(x**p))**(1/p) 
#参考　https://qiita.com/hkthirano/items/9750290cb34a3e4570f1

実装したドロップアウトクラスと、パラメータノルムペナルティを使って、ドロップアウトとL2ノルム正則化（重み減衰）を含めたNNを実装してみます。

[重要!!!]
- 損失関数に $ \frac{\lambda}{2}\|w\|_2^2 $を正則化項として加える。

In [10]:
class MLP_classifier():

    def __init__(self, weight_decay_lambda=0):
        '''
        構造
        x -> fc(783, 256) -> relu -> dropout -> fc(256, 256) -> relu -> dropout -> fc(256, 10) -> out
        '''
        
        # 層の定義
        self.fc1 = FullyConnectedLayer(784, 256)
        self.relu1 = ReLU()
        self.dropout1 = Dropout()
        self.fc2 = FullyConnectedLayer(256, 256)
        self.relu2 = ReLU()
        self.dropout2 = Dropout()
        self.fc3 = FullyConnectedLayer(256, 10)
        self.out = None
        
        # 損失関数の定義
        self.criterion = SoftmaxCrossEntropyLoss()
        self.weight_decay_lambda = weight_decay_lambda

    def forward(self, x, train_flg=True):
        '''
        順伝播
        '''
        
        x = self.relu1(self.fc1(x))
        x = self.dropout1(x, train_flg)
        x = self.relu2(self.fc2(x))
        x = self.dropout2(x, train_flg)
        self.out = self.fc3(x)
        
        # 勾配計算の都合上softmaxはこの順伝播関数内では行わない
        # 予測するときはさらにsoftmaxを通す必要がある
        return self.out
    
    def loss(self, x, t):
        loss = self.criterion(t, self.forward(x))
        for fc in [self.fc1, self.fc2, self.fc3]:
            loss += (self.weight_decay_lambda/2) * lp_norm(fc.w, p=2)**2 ##重要　ここで正則化項を罰則として与えている##
        return loss

    def backward(self):
        '''
        逆伝播
        '''
        # 勾配を逆伝播
        d = self.criterion.backward()
        d = self.fc3.backward(d)
        d = self.dropout2.backward(d)
        d = self.relu2.backward(d)
        d = self.fc2.backward(d)
        d = self.dropout1.backward(d)
        d = self.relu1.backward(d)
        d = self.fc1.backward(d)
        
        for fc in [self.fc1, self.fc2, self.fc3]:
            fc.dw += self.weight_decay_lambda * fc.w ##重要##

    def optimize_GradientDecent(self, lr):
        '''
        勾配降下法による全層のパラメータの更新
        '''
        for fc in [self.fc1, self.fc2, self.fc3]:
            fc.w -= lr * fc.dw
            fc.b -= lr * fc.db

### 学習
train_loss が1.4前後、accuracyが80%前後の結果になりました。より精度を高める場合には、パラメータ最適化法などが考えられると思いますが、今回の実装演習では精度が確保できました。

In [11]:
# モデルの宣言
model = MLP_classifier(weight_decay_lambda=0.1)

# 学習率
lr = 0.02
# 学習エポック数
n_epoch = 30

# n_epoch繰り返す
for n in range(n_epoch):
    loss = model.loss(X_train, T_train)    
    model.backward()
    
    model.optimize_GradientDecent(lr)
        
    # テスト
    y = model.forward(X_test, train_flg=False)
    pred = softmax(y)
    accuracy = np.mean(np.equal(np.argmax(y, axis=1), np.argmax(T_test, axis=1)))
    
    print(f'EPOCH {n + 1} | TRAIN LOSS {loss:.5f} | ACCURACY {accuracy:.2%}')

EPOCH 1 | TRAIN LOSS 1.57379 | ACCURACY 19.41%
EPOCH 2 | TRAIN LOSS 1.56571 | ACCURACY 29.18%
EPOCH 3 | TRAIN LOSS 1.55825 | ACCURACY 37.47%
EPOCH 4 | TRAIN LOSS 1.55084 | ACCURACY 43.16%
EPOCH 5 | TRAIN LOSS 1.54349 | ACCURACY 47.99%
EPOCH 6 | TRAIN LOSS 1.53618 | ACCURACY 51.86%
EPOCH 7 | TRAIN LOSS 1.52864 | ACCURACY 55.35%
EPOCH 8 | TRAIN LOSS 1.52106 | ACCURACY 58.36%
EPOCH 9 | TRAIN LOSS 1.51311 | ACCURACY 61.25%
EPOCH 10 | TRAIN LOSS 1.50461 | ACCURACY 63.52%
EPOCH 11 | TRAIN LOSS 1.49584 | ACCURACY 65.43%
EPOCH 12 | TRAIN LOSS 1.48692 | ACCURACY 67.10%
EPOCH 13 | TRAIN LOSS 1.47694 | ACCURACY 68.45%
EPOCH 14 | TRAIN LOSS 1.46680 | ACCURACY 69.31%
EPOCH 15 | TRAIN LOSS 1.45661 | ACCURACY 70.41%
EPOCH 16 | TRAIN LOSS 1.44611 | ACCURACY 71.56%
EPOCH 17 | TRAIN LOSS 1.43593 | ACCURACY 72.55%
EPOCH 18 | TRAIN LOSS 1.42626 | ACCURACY 73.66%
EPOCH 19 | TRAIN LOSS 1.41661 | ACCURACY 74.61%
EPOCH 20 | TRAIN LOSS 1.40783 | ACCURACY 75.64%
EPOCH 21 | TRAIN LOSS 1.39989 | ACCURACY 76.33%
E